In [1]:
import torch
import os
from datetime import datetime

# Import your custom modules
from config.Experiment_Config import (
    ExperimentConfig,
    ModelSettings,
    TrainingSettings,
    DataSettings,
    ModelSelection,
    TokenizerSettings
)
from models.LSTM.BiLSTM import BiLSTM
from models.LSTM.CNNBiLSTM import CNNBiLSTM
from utils.load_split import loader
from exp.trainer import Trainer
import matplotlib.pyplot as plt
import joblib


/Users/yunkaigao/opt/anaconda3/envs/multi_turn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_settings = ModelSettings(
    input_dim=768,  # For BERT base
    output_dim=3,   # Number of classes
    hidden_dims=[512, 256, 128],
    embedding_dim=768,
    embedding_type='BERT_base_uncased',
    lags=5,
    dropout_rate=0.2
)

In [3]:
training_settings = TrainingSettings(
    num_epochs=100,
    batch_size=32,
    learning_rate=0.001,
    weight_decay=0.01,
    gradient_clip=1.0,
    early_stopping_patience=5,
    save_model_dir='saved_models',
    save_results_dir='results'
)


In [4]:
data_settings = DataSettings(
    which="question",
    train_size=0.8,
    val_size=0.1,
    test_size=0.1,
    batch_size=32,
    shuffle=True,
    drop_last=True
)

In [5]:
model_selection = ModelSelection(
    model_type='CNNBiLSTM',
    use_cnn=True,
    cnn_layers=2
)

In [6]:
tokenizer_settings = TokenizerSettings(
    name="BERT_base_uncased",
    embedding_type="bert",
    truncation="ratio",  # or "equal"
    max_length=512,
    padding=True,
    add_special_tokens=True
)

In [7]:
config = ExperimentConfig(
    model_settings=model_settings,
    training_settings=training_settings,
    data_settings=data_settings,
    model_selection=model_selection,
    tokenizer_settings=tokenizer_settings
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.training_settings.device = device

print(f"Using device: {device}")
print(f"Model type: {config.model_selection.model_type}")

Using device: cpu
Model type: CNNBiLSTM


In [9]:
train_loader, val_loader, test_loader, label_encoder = loader(config)
joblib.dump(label_encoder, 'label_encoder.pkl')
num_classes = len(label_encoder.classes_)

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


In [10]:
if config.model_selection.model_type == 'BiLSTM':
    model = BiLSTM(config.model_settings)
elif config.model_selection.model_type == 'CNNBiLSTM':
    model = CNNBiLSTM(config.model_settings)

model = model.to(device)

In [11]:
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    config=config
)

# Train model
print("Starting training...")
metrics_history = trainer.train()

Starting training...
Training on device: cpu

Epoch 1/100 - train


Training:   0%|          | 0/127 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Training:   0%|          | 0/127 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not str

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(trainer.train_losses, label='Training Loss')
plt.plot(trainer.val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Print final test results
print("\nTest Results:")
test_metrics, _, _ = trainer.evaluate(test_loader, 'test')
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")